# HPVsim interventions demonstration

In this tutorial we will walk through how to set up a simulation, define some parameters, and compare different interventions. This will rely upon *interventions*, which are a standard class and can be easily customized by users.

Let's start by defining base parameters for a simulation:

In [ ]:
import hpvsim as hpv

base_pars = dict(
    n_agents=10e3,
    start=1950,
    end=2060,
    genotypes=[16, 18, 'hi5', 'ohr'],
    location='india',
    verbose=0.0
)


## Define interventions to simulate

In [ ]:
    ### Create interventions
    routine_vx_dose1 = hpv.routine_vx(
        prob = 0.9,
        start_year = 2023,
        age_range = [9,10],
        product = 'bivalent',
        label = 'Bivalent dose 1'
    )

    second_dose_eligible = lambda sim: (sim.people.doses == 1) & (sim.t > (sim.people.date_vaccinated + 0.5 / sim['dt']))
    routine_vx_dose2 = hpv.routine_vx(
        prob = 0.9,
        start_year = 2023,
        product = 'bivalent2',
        eligibility=second_dose_eligible,
        label = 'Bivalent dose 2'
    )
    
    catchup_campaign_vx = hpv.campaign_vx(
        prob=0.9,
        years=2023,
        product='bivalent',
        age_range=(9, 24),
        label='Catch-up campaign'
    )
    
    routine_vx = [routine_vx_dose1, routine_vx_dose2]
    catchup_vx = [catchup_campaign_vx]
    

In [ ]:
    scenarios = {
        'baseline': {
            'name':'No interventions',
            'pars': {
                
            }
        },
        'vx_scaleup': {
            'name':'Vaccinate 90% of 9yo',
            'pars': {
                'interventions': routine_vx
            }
        },
        'vx_scaleup_MAC': {
            'name':'Vaccinate 90% of 9-24yo',
            'pars': {
                'interventions': routine_vx + catchup_vx
            }
        },
    }
    
    base_sim = hpv.Sim(pars=base_pars)
    metapars = {'n_runs': 3}
    scens = hpv.Scenarios(sim=base_sim, metapars=metapars, scenarios=scenarios)
    scens.run()

    print('Done running scenarios')
    

## Plotting
Now let's plot some basic information to see what we just ran.

In [ ]:
    to_plot = {
        'Age standardized cancer incidence (per 100,000 women)': ['asr_cancer_incidence'],
        'Cancer deaths per 100,000 women': ['cancer_mortality'],
    }
    scens.plot(to_plot=to_plot)

In [ ]:
    # Define a nonvalent vaccine
    nonavalent_vx_dose1 = hpv.routine_vx(
        prob = 0.9,
        start_year = 2023,
        age_range = [9,10],
        product = 'nonvalent',
        label = 'Nonavalent dose 1'
    )
    
    # Define a gender-neutral vaccine strategy
    gender_neutral_vx_dose1 = hpv.routine_vx(
        prob = 0.9,
        start_year = 2023,
        age_range = [9,10],
        sex = [0,1],
        product = 'bivalent',
        label = 'Bivalent dose 1'
    )